# Machine Learning - Andrew Ng ( Python Implementation)

## Anomaly Detection

### Loading the Data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [ ]:
mat = loadmat("ex8data1.mat")
X = mat["X"]
Xval = mat["Xval"]
yval = mat["yval"]

In [ ]:
X

In [ ]:
yval

## Diabete data loading

In [65]:
log_processing(3)

1.0986122886681098

In [16]:
f = open('diabetes.csv','r')
a = f.readline()
data = []
dv = []
for line in f.readlines():
    l = line.strip().split(',')
    data.append(l[:-1])
    dv.append([l[-1]])
f.close()
for i in range(len(data)):
    buff = []
    for j in range(len(data[i])):
        n = float(data[i][j])
        if j in [0,3,4,6,7]:
            buff.append(np.log(n+0.01))
        else:
            buff.append(n)
    data[i] = buff
print(data)

[[1.7934247485471162, 148.0, 72.0, 3.555633734966574, -4.605170185988091, 33.6, -0.45098562340997367, 3.9122229854308124], [0.009950330853168092, 85.0, 66.0, 3.367640598133313, -4.605170185988091, 26.6, -1.018877320649256, 3.4343097331123578], [2.080690761080268, 183.0, 64.0, -4.605170185988091, -4.605170185988091, 23.3, -0.38272562113867487, 3.4660483539817717], [0.009950330853168092, 89.0, 66.0, 3.1359289040472746, 4.543401159590459, 28.1, -1.7316055464083078, 3.044998514856909], [-4.605170185988091, 137.0, 40.0, 3.555633734966574, 5.124023501441311, 43.1, 0.8320391794265638, 3.4968105458651015], [1.6114359150967734, 116.0, 74.0, -4.605170185988091, -4.605170185988091, 25.6, -1.5558971455060704, 3.4015306594522756], [1.1019400787607843, 78.0, 50.0, 3.4660483539817717, 4.47745044438572, 31.0, -1.3547956940605197, 3.25848107946056], [2.303584593327129, 115.0, 0.0, -4.605170185988091, -4.605170185988091, 35.3, -1.9379419794061363, 3.367640598133313], [0.6981347220709843, 197.0, 70.0, 3.

In [17]:
d = np.asarray(data,dtype=np.float64)
dval = np.asarray(dv,dtype=np.int64)

In [31]:
d[0]

array([  1.79342475, 148.        ,  72.        ,   3.55563373,
        -4.60517019,  33.6       ,  -0.45098562,   3.91222299])

### Plotting the data

In [ ]:
plt.scatter(X[:,0],X[:,1],marker="x")
plt.xlim(0,30)
plt.ylim(0,30)
plt.xlabel("Latency (ms)")
plt.ylabel("Throughput (mb/s)")

### Gaussian Distribution

$p(x;\mu,\sigma^2) = \frac{1}{\sqrt{2\pi\sigma^2}}e^{\frac{(x-\mu)^2}{2\sigma^2}}$

$\mu_i = \frac{1}{m}\sum^m_{j=1}x^{(j)}$

$\sigma^2_i = \frac{1}{m}\sum^m_{j=1}(x^{(j)} - \mu_j)^2$

In [19]:
def estimateGaussian(X):
    """
     This function estimates the parameters of a Gaussian distribution using the data in X
    """
    
    m = X.shape[0]
    
    #compute mean
    sum_ = np.sum(X,axis=0)
    mu = 1/m *sum_
    
    # compute variance
    var = 1/m * np.sum((X - mu)**2,axis=0)
    
    return mu,var

In [ ]:
mu, sigma2 = estimateGaussian(X)

In [20]:
mud, sigmad = estimateGaussian(d)

In [21]:
mud, sigmad

(array([  0.3768975 , 120.89453125,  69.10546875,   0.965687  ,
          0.22405576,  31.99257812,  -0.92876214,   3.44913581]),
 array([4.73709376e+00, 1.02091726e+03, 3.74159449e+02, 1.31282094e+01,
        2.23874932e+01, 6.20790465e+01, 3.91237829e-01, 1.03938957e-01]))

### Multivariate Gaussian Distribution

$p(x;\mu,\Sigma) = \frac{1}{(2\pi)^{n/2}|\Sigma|^{1/2}} exp(-\frac{1}{2}(x-\mu)^T\Sigma^{-1}(x-\mu))$

In [22]:
def multivariateGaussian(X, mu, sigma2):
    """
    Computes the probability density function of the multivariate gaussian distribution.
    """
    k = len(mu)
    
    sigma2=np.diag(sigma2)
    X = X - mu.T
    p = 1/((2*np.pi)**(k/2)*(np.linalg.det(sigma2)**0.5))* np.exp(-0.5* np.sum(X @ np.linalg.pinv(sigma2) * X,axis=1))
    return p

In [ ]:
p = multivariateGaussian(X, mu, sigma2)

In [23]:
p1 = multivariateGaussian(d, mud, sigmad)

In [32]:
p1

array([1.17296627e-09, 3.38205091e-09, 1.27292301e-10, 1.01004751e-09,
       1.14225646e-12, 1.11931253e-09, 1.43552348e-09, 8.65522651e-13,
       3.41588445e-11, 4.21277463e-14, 5.74484925e-10, 4.46862901e-10,
       2.71930155e-11, 7.48375758e-11, 5.22817228e-10, 3.04858934e-12,
       6.88754558e-11, 1.67925980e-09, 1.80289323e-10, 7.42080894e-09,
       1.53320997e-09, 4.48056702e-10, 3.67070644e-11, 3.65967625e-09,
       4.12135962e-10, 2.68745895e-09, 5.46166523e-10, 1.89268893e-09,
       2.31899010e-10, 1.06468733e-09, 6.58967230e-10, 1.41075417e-09,
       1.69006521e-09, 1.33451436e-10, 2.09891431e-09, 1.11221337e-09,
       1.56234528e-09, 1.52172756e-09, 2.88405312e-09, 1.27665611e-10,
       1.16275909e-09, 5.66508443e-10, 4.89891114e-10, 6.15729102e-12,
       6.83219553e-10, 1.50540231e-12, 1.42602296e-09, 9.45639816e-10,
       3.49223031e-09, 6.36405385e-16, 1.10471028e-09, 2.76278853e-09,
       3.71689915e-09, 1.00038860e-10, 1.27276128e-09, 3.60376882e-10,
      

### Visualize the fit

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(X[:,0],X[:,1],marker="x")
X1,X2 = np.meshgrid(np.linspace(0,35,num=70),np.linspace(0,35,num=70))
p2 = multivariateGaussian(np.hstack((X1.flatten()[:,np.newaxis],X2.flatten()[:,np.newaxis])), mu, sigma2)
contour_level = 10**np.array([np.arange(-20,0,3,dtype=np.float)]).T
plt.contour(X1,X2,p2[:,np.newaxis].reshape(X1.shape),contour_level)
plt.xlim(0,35)
plt.ylim(0,35)
plt.xlabel("Latency (ms)")
plt.ylabel("Throughput (mb/s)")

### Selecting the threshold $\epsilon$

In [24]:
def selectThreshold(yval, pval):
    """
    Find the best threshold (epsilon) to use for selecting outliers
    """
    best_epi = 0
    best_F1 = 0
    
    stepsize = (max(pval) -min(pval))/1000
    epi_range = np.arange(pval.min(),pval.max(),stepsize)
    for epi in epi_range:
        predictions = (pval<epi)[:,np.newaxis]
        tp = np.sum(predictions[yval==1]==1)
        fp = np.sum(predictions[yval==0]==1)
        fn = np.sum(predictions[yval==1]==0)
        
        # compute precision, recall and F1
        prec = tp/(tp+fp)
        rec = tp/(tp+fn)
        
        F1 = (2*prec*rec)/(prec+rec)
        
        if F1 > best_F1:
            best_F1 =F1
            best_epi = epi
        
    return best_epi, best_F1

In [ ]:
pval = multivariateGaussian(Xval, mu, sigma2)
epsilon, F1 = selectThreshold(yval, pval)
print("Best epsilon found using cross-validation:",epsilon)
print("Best F1 on Cross Validation Set:",F1)

In [25]:
epsilon1, F1d = selectThreshold(dval, p1)

/home/andrew1993/.conda/envs/tutorial_t/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in long_scalars


In [26]:
epsilon1

1.5265495893881898e-09

In [27]:
F1d

0.5429638854296389

In [28]:
outliers = np.nonzero(p1<epsilon1)[0]
print(outliers)

[  0   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  18  21
  22  24  26  28  29  30  31  33  35  37  39  40  41  42  43  44  45  46
  47  49  50  53  54  55  56  57  58  59  60  61  62  64  66  67  72  75
  76  78  80  81  83  84  86  88  90  92  93  94  96  97  99 100 101 102
 104 105 106 109 110 111 113 114 115 116 117 119 120 123 124 125 129 131
 135 136 137 138 140 143 145 146 147 148 149 151 152 153 154 155 156 157
 159 161 162 164 170 172 173 175 176 177 178 179 180 181 182 183 184 185
 186 187 190 192 193 194 195 196 199 200 201 202 204 206 207 209 211 212
 213 215 218 219 220 221 222 223 225 226 227 228 229 230 231 232 235 236
 237 238 239 240 242 243 245 246 247 249 250 253 254 257 258 259 260 261
 263 265 266 267 268 269 270 271 272 273 274 277 278 279 280 283 284 285
 286 287 290 291 292 293 294 297 298 299 300 302 303 304 306 307 308 310
 311 314 317 319 322 323 324 325 327 328 329 330 332 333 335 336 337 338
 339 342 344 345 346 347 349 350 351 352 354 355 35

In [29]:
dval[outliers]

array([[1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
    

In [30]:
po = 0
ne = 0
for cls in dval[outliers]:
    if cls[0]== 1:
        po +=1
    else:
        ne +=1
print(po,ne)

218 317


### Visualize the anomalies 

In [ ]:
plt.figure(figsize=(8,6))

# plot the data
plt.scatter(X[:,0],X[:,1],marker="x")

# potting of contour
X1,X2 = np.meshgrid(np.linspace(0,35,num=70),np.linspace(0,35,num=70))
p2 = multivariateGaussian(np.hstack((X1.flatten()[:,np.newaxis],X2.flatten()[:,np.newaxis])), mu, sigma2)
contour_level = 10**np.array([np.arange(-20,0,3,dtype=np.float)]).T
plt.contour(X1,X2,p2[:,np.newaxis].reshape(X1.shape),contour_level)

# Circling of anomalies
outliers = np.nonzero(p<epsilon)[0]
plt.scatter(X[outliers,0],X[outliers,1],marker ="o",facecolor="none",edgecolor="r",s=70)

plt.xlim(0,35)
plt.ylim(0,35)
plt.xlabel("Latency (ms)")
plt.ylabel("Throughput (mb/s)")

## High Dimensional Dataset

### Loading the data

In [ ]:
mat2 = loadmat("ex8data2.mat")
X2 = mat2["X"]
Xval2 = mat2["Xval"]
yval2 = mat2["yval"]

In [ ]:
X2

In [ ]:
# compute the mean and variance
mu2, sigma2_2 = estimateGaussian(X2)

In [ ]:
# Training set
p3 = multivariateGaussian(X2, mu2, sigma2_2)

# cross-validation set
pval2 = multivariateGaussian(Xval2, mu2, sigma2_2)

# Find the best threshold
epsilon2, F1_2 = selectThreshold(yval2, pval2)
print("Best epsilon found using cross-validation:",epsilon2)
print("Best F1 on Cross Validation Set:",F1_2)
print("# Outliers found:",np.sum(p3<epsilon2))

## Recommender Systems

### Loading of data

In [ ]:
mat3 = loadmat("ex8_movies.mat")
mat4 = loadmat("ex8_movieParams.mat")
Y = mat3["Y"] # 1682 X 943 matrix, containing ratings (1-5) of 1682 movies on 943 user
R = mat3["R"] # 1682 X 943 matrix, where R(i,j) = 1 if and only if user j give rating to movie i
X = mat4["X"] # 1682 X 10 matrix , num_movies X num_features matrix of movie features
Theta = mat4["Theta"] # 943 X 10 matrix, num_users X num_features matrix of user features

In [ ]:
# Compute average rating 
print("Average rating for movie 1 (Toy Story):",np.sum(Y[0,:]*R[0,:])/np.sum(R[0,:]),"/5")

### Visualize the ratings matrix

In [ ]:
plt.figure(figsize=(8,16))
plt.imshow(Y)
plt.xlabel("Users")
plt.ylabel("Movies")

### Collaborative Filtering Learning Algorithm

$J(x^{(i)},...,x^{(n_m)},\Theta^{(I)},...,\Theta^{(n_u)}) = \frac{1}{2} \sum ((\Theta^{(j)})^Tx^{(i)} - y^{(i,j)})^2 + (\frac{\lambda}{2} \sum^{n_u}_{j=1}\sum^n_{k=1} (\Theta^{(j)}_k)^2) + (\frac{\lambda}{2} \sum^{n_m}_{i=1}\sum^n_{k=1} (x^{(i)}_k)^2)$

$\frac{\partial J}{\partial x^{(i)}_k} = \sum ((\Theta^{(j)})^Tx^{(i)} - y^{(i,j)})\Theta_k^{(j)} +\lambda x^{(i)}_k$

$\frac{\partial J}{\partial \Theta^{(j)}_k} = \sum ((\Theta^{(j)})^Tx^{(i)} - y^{(i,j)})x_k^{(i)} +\lambda \Theta^{(j)}_k$

In [ ]:
def  cofiCostFunc(params, Y, R, num_users, num_movies, num_features, Lambda):
    """
    Returns the cost and gradient for the collaborative filtering problem
    """
    
    # Unfold the params
    X = params[:num_movies*num_features].reshape(num_movies,num_features)
    Theta = params[num_movies*num_features:].reshape(num_users,num_features)
    
    predictions =  X @ Theta.T
    err = (predictions - Y)
    J = 1/2 * np.sum((err**2) * R)
    
    #compute regularized cost function
    reg_X =  Lambda/2 * np.sum(Theta**2)
    reg_Theta = Lambda/2 *np.sum(X**2)
    reg_J = J + reg_X + reg_Theta
    
    # Compute gradient
    X_grad = err*R @ Theta
    Theta_grad = (err*R).T @ X
    grad = np.append(X_grad.flatten(),Theta_grad.flatten())
    
    # Compute regularized gradient
    reg_X_grad = X_grad + Lambda*X
    reg_Theta_grad = Theta_grad + Lambda*Theta
    reg_grad = np.append(reg_X_grad.flatten(),reg_Theta_grad.flatten())
    
    return J, grad, reg_J, reg_grad

In [ ]:
# Reduce the data set size to run faster
num_users, num_movies, num_features = 4,5,3
X_test = X[:num_movies,:num_features]
Theta_test= Theta[:num_users,:num_features]
Y_test = Y[:num_movies,:num_users]
R_test = R[:num_movies,:num_users]
params = np.append(X_test.flatten(),Theta_test.flatten())

# Evaluate cost function
J, grad = cofiCostFunc(params, Y_test, R_test, num_users, num_movies, num_features, 0)[:2]
print("Cost at loaded parameters:",J)

J2, grad2 = cofiCostFunc(params, Y_test, R_test, num_users, num_movies, num_features, 1.5)[2:]
print("Cost at loaded parameters (lambda = 1.5):",J2)

### Learning movie recommendations

In [ ]:
# load movie list
movieList = open("movie_ids.txt","r").read().split("\n")[:-1]

# see movie list
np.set_printoptions(threshold=np.nan)
movieList

In [ ]:
# Initialize my ratings
my_ratings = np.zeros((1682,1))

# Create own ratings
my_ratings[0] = 4 
my_ratings[97] = 2
my_ratings[6] = 3
my_ratings[11]= 5
my_ratings[53] = 4
my_ratings[63]= 5
my_ratings[65]= 3
my_ratings[68] = 5
my_ratings[82]= 4
my_ratings[225] = 5
my_ratings[354]= 5

print("New user ratings:\n")
for i in range(len(my_ratings)):
    if my_ratings[i]>0:
        print("Rated",int(my_ratings[i]),"for index",movieList[i])

### Recommendations

In [ ]:
def normalizeRatings(Y, R):
    """
    normalized Y so that each movie has a rating of 0 on average, and returns the mean rating in Ymean.
    """
    
    m,n = Y.shape[0], Y.shape[1]
    Ymean = np.zeros((m,1))
    Ynorm = np.zeros((m,n))
    
    for i in range(m):
        Ymean[i] = np.sum(Y[i,:])/np.count_nonzero(R[i,:])
        Ynorm[i,R[i,:]==1] = Y[i,R[i,:]==1] - Ymean[i]
        
    return Ynorm, Ymean

In [ ]:
def gradientDescent(initial_parameters,Y,R,num_users,num_movies,num_features,alpha,num_iters,Lambda):
    """
    Optimize X and Theta
    """
    # unfold the parameters
    X = initial_parameters[:num_movies*num_features].reshape(num_movies,num_features)
    Theta = initial_parameters[num_movies*num_features:].reshape(num_users,num_features)
    
    J_history =[]
    
    for i in range(num_iters):
        params = np.append(X.flatten(),Theta.flatten())
        cost, grad = cofiCostFunc(params, Y, R, num_users, num_movies, num_features, Lambda)[2:]
        
        # unfold grad
        X_grad = grad[:num_movies*num_features].reshape(num_movies,num_features)
        Theta_grad = grad[num_movies*num_features:].reshape(num_users,num_features)
        X = X - (alpha * X_grad)
        Theta = Theta - (alpha * Theta_grad)
        J_history.append(cost)
    
    paramsFinal = np.append(X.flatten(),Theta.flatten())
    return paramsFinal , J_history

In [ ]:
Y = np.hstack((my_ratings,Y))
R =np.hstack((my_ratings!=0,R))

# Normalize Ratings
Ynorm, Ymean = normalizeRatings(Y, R)

In [ ]:
num_users = Y.shape[1]
num_movies = Y.shape[0]
num_features = 10

# Set initial Parameters (Theta,X)
X = np.random.randn(num_movies, num_features)
Theta = np.random.randn(num_users, num_features)
initial_parameters = np.append(X.flatten(),Theta.flatten())
Lambda = 10

# Optimize parameters using Gradient Descent
paramsFinal, J_history = gradientDescent(initial_parameters,Y,R,num_users,num_movies,num_features,0.001,400,Lambda)

### Ploting of cost Function

In [ ]:
plt.plot(J_history)
plt.xlabel("Iteration")
plt.ylabel("$J(\Theta)$")
plt.title("Cost function using Gradient Descent")

In [ ]:
# unfold paramaters
X = paramsFinal[:num_movies*num_features].reshape(num_movies,num_features)
Theta = paramsFinal[num_movies*num_features:].reshape(num_users,num_features)

# Predict rating
p = X @ Theta.T
my_predictions = p[:,0][:,np.newaxis] + Ymean

In [ ]:
import pandas as pd
df = pd.DataFrame(np.hstack((my_predictions,np.array(movieList)[:,np.newaxis])))
df.sort_values(by=[0],ascending=False,inplace=True)
df.reset_index(drop=True,inplace=True)

In [ ]:
print("Top recommendations for you:\n")
for i in range(10):
    print("Predicting rating",round(float(df[0][i]),1)," for index",df[1][i])